<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/overview_Jan9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
!pip install datasets -q


In [59]:
import os
import glob
import random
from PIL import Image
from torch.utils.data import Dataset,random_split
from datasets import Dataset, DatasetDict
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [61]:
model_name="caltech256-xlstm"

from google.colab import drive
drive.mount('/content/drive')

import os

CHECKPOINTS_DIR = '/content/drive/MyDrive/ckpts'

def save_checkpoint(model, optimizer, epoch):
    os.makedirs(CHECKPOINTS_DIR, exist_ok=True)
    checkpoint_path = f'{CHECKPOINTS_DIR}/{model_name}.pth'
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, checkpoint_path)
    print(f"ckpt saved for {model_name} at epoch {epoch}.")

def load_checkpoint(model, optimizer):
    ckpt_path = f'{CHECKPOINTS_DIR}/{model_name}.pth'
    if not os.path.exists(ckpt_path):
        print(f"no ckpt found for {model_name} starting from epoch 0.")
        return 0

    checkpoint = torch.load(ckpt_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"ckpt loaded for {model_name} from {ckpt_path}. resuming from epoch {start_epoch}.")

    return start_epoch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("jessicali9530/caltech256")
print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/jessicali9530/caltech256/versions/2


In [63]:
import os
import random
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import glob
from torchvision import transforms

def set_style():
    """Set the style for prettier visualizations"""
    plt.style.use('default')  # Using default style instead of seaborn
    plt.rcParams['figure.facecolor'] = 'white'
    plt.rcParams['axes.facecolor'] = 'white'
    plt.rcParams['figure.dpi'] = 150

def create_random_mask(size, mask_ratio=0.05):  # Reduced mask ratio to 5%
    """
    Create multiple small random masks
    """
    height, width = size
    mask = np.ones((height, width), np.float32)

    # Create 2-4 small masks instead of one large mask
    num_masks = random.randint(2, 4)

    for _ in range(num_masks):
        # Calculate small rectangle dimensions
        mask_height = int(height * np.sqrt(mask_ratio))
        mask_width = int(width * np.sqrt(mask_ratio))

        # Make masks even smaller with random scaling
        scale = random.uniform(0.3, 0.7)
        mask_height = int(mask_height * scale)
        mask_width = int(mask_width * scale)

        # Calculate random position
        top = random.randint(0, height - mask_height)
        left = random.randint(0, width - mask_width)

        # Apply mask
        mask[top:top + mask_height, left:left + mask_width] = 0

    return mask

def process_and_visualize_masked_images(data_path, num_samples=5, target_size=(256, 256)):
    """
    Process images with masks and visualize results with enhanced styling
    """
    # Set the style
    set_style()

    # Get image paths
    image_paths = []
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(root, file))

    # Select random images
    selected_images = random.sample(image_paths, min(num_samples, len(image_paths)))

    # Setup transforms
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])

    # Create figure with better spacing
    fig = plt.figure(figsize=(20, 4))
    plt.subplots_adjust(wspace=0.3)

    for idx, img_path in enumerate(selected_images):
        # Load and transform image
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img)

        # Create mask
        mask = create_random_mask(target_size)
        mask_tensor = torch.from_numpy(mask).unsqueeze(0)

        # Apply mask to image
        masked_img = img_tensor * mask_tensor

        # Convert tensors back to numpy for visualization
        orig_img = img_tensor.permute(1, 2, 0).numpy()
        mask_viz = mask_tensor.permute(1, 2, 0).numpy()
        masked_img = masked_img.permute(1, 2, 0).numpy()

        # Create subplots in a vertical arrangement
        ax1 = plt.subplot(3, num_samples, idx + 1)
        ax2 = plt.subplot(3, num_samples, idx + 1 + num_samples)
        ax3 = plt.subplot(3, num_samples, idx + 1 + 2*num_samples)

        # Plot with enhanced styling
        ax1.imshow(orig_img)
        ax1.set_title('Original', pad=5)

        ax2.imshow(mask_viz, cmap='cool')  # Changed mask colormap
        ax2.set_title('Mask', pad=5)

        ax3.imshow(masked_img)
        ax3.set_title('Masked', pad=5)

        # Remove axes for cleaner look
        for ax in [ax1, ax2, ax3]:
            ax.set_xticks([])
            ax.set_yticks([])
            ax.spines['top'].set_visible(False)
            ax.spines['right'].set_visible(False)
            ax.spines['bottom'].set_visible(False)
            ax.spines['left'].set_visible(False)

    plt.tight_layout()
    plt.show()

def create_masked_dataset(data_path, num_images=1000, target_size=(256, 256)):
    """
    Create a dataset of original and masked images
    """
    # Get all image paths
    image_paths = []
    for root, dirs, files in os.walk(data_path):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(root, file))

    # Select random images
    selected_images = random.sample(image_paths, min(num_images, len(image_paths)))

    # Setup transforms
    transform = transforms.Compose([
        transforms.Resize(target_size),
        transforms.ToTensor()
    ])

    dataset = []
    for img_path in selected_images:
        # Load and transform image
        img = Image.open(img_path).convert('RGB')
        img_tensor = transform(img)

        # Create mask
        mask = create_random_mask(target_size)
        mask_tensor = torch.from_numpy(mask).unsqueeze(0)

        # Apply mask to image
        masked_img = img_tensor * mask_tensor

        dataset.append({
            'original': img_tensor,
            'masked': masked_img,
            'mask': mask_tensor
        })

    return dataset